In [ ]:
import numpy as np
import pandas as 
import random
from sklearn import preprocessing
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
import os
from sklearn.metrics.pairwise import cosine_similarity
import re
from scipy.sparse import csr_matrix
import scipy as sp
from gensim.models import Word2Vec
import gensim
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(font_scale=1)
sns.set_style("white")
pd.set_option('display.max_columns', 37)
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display_html 
from IPython.core.display import HTML
from collections import defaultdict
from surprise.model_selection import train_test_split
# Packages for model evaluation
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from time import time

# Package to suppress warnings
import warnings
warnings.filterwarnings("ignore")
# Packages for saving models
import pickle
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.impute import SimpleImputer
import scipy as sp
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from gensim.models import Word2Vec
from surprise.model_selection import cross_validate
import scipy.stats


In [ ]:
train.head()

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/edsa-movie-recommender/sample_submission.csv')
movies = pd.read_csv('/kaggle/input/edsa-movie-recommender/movies.csv')
imdb_data = pd.read_csv('/kaggle/input/edsa-movie-recommender/imdb_data.csv')
genome_scores = pd.read_csv('/kaggle/input/edsa-movie-recommender/genome_scores.csv')
genome_tags = pd.read_csv('/kaggle/input/edsa-movie-recommender/genome_tags.csv')
train = pd.read_csv('/kaggle/input/edsa-movie-recommender/train.csv')
test = pd.read_csv('/kaggle/input/edsa-movie-recommender/test.csv')
tags = pd.read_csv('/kaggle/input/edsa-movie-recommender/tags.csv')
links = pd.read_csv('/kaggle/input/edsa-movie-recommender/links.csv')

In [ ]:
# Number of users
print('The ratings dataset has', train['userId'].nunique(), 'unique users')
# Number of movies
print('The ratings dataset has', train['movieId'].nunique(), 'unique movies')
# Number of ratings
print('The ratings dataset has', train['rating'].nunique(), 'unique ratings')
# List of unique ratings
print('The unique ratings are', sorted(train['rating'].unique()))

In [ ]:
# Merge ratings and movies datasets
df = pd.merge(train, movies, on='movieId', how='inner')
# Take a look at the data
df.head()

In [ ]:
# Aggregate by movie
agg_ratings = df.groupby('title').agg(mean_rating = ('rating', 'mean'),
                                                number_of_ratings = ('rating', 'count')).reset_index()
# Keep the movies with over 100 ratings
agg_ratings_GT100 = agg_ratings[agg_ratings['number_of_ratings']>100]
agg_ratings_GT100.info()

In [ ]:
# Check popular movies
agg_ratings_GT100.sort_values(by='number_of_ratings', ascending=False).head()

In [ ]:
# Visulization
sns.jointplot(x='mean_rating', y='number_of_ratings', data=agg_ratings_GT100)

In [ ]:
# Merge data
df_GT100 = pd.merge(df, agg_ratings_GT100[['title']], on='title', how='inner')
df_GT100.info()

In [ ]:
# Number of users
print('The ratings dataset has', df_GT100['userId'].nunique(), 'unique users')
# Number of movies
print('The ratings dataset has', df_GT100['movieId'].nunique(), 'unique movies')
# Number of ratings
print('The ratings dataset has', df_GT100['rating'].nunique(), 'unique ratings')
# List of unique ratings
print('The unique ratings are', sorted(df_GT100['rating'].unique()))


In [ ]:
import random


# Calculate the number of rows in the DataFrame
n = df_GT100.shape[0]

# Calculate the sample size (0.01 percent of the original
s = n // 10000

# Generate the list of row indices to skip while sampling the DataFrame
skip_indices = random.sample(range(n), n - s)

# Sample the DataFrame by dropping the rows specified by the skip_indices list
sampled_df = df_GT100.drop(skip_indices)

# Number of users in the sampled dataframe
print('The sampled dataframe has', sampled_df['userId'].nunique(), 'unique users')

# Number of movies in the sampled dataframe
print('The sampled dataframe has', sampled_df['movieId'].nunique(), 'unique movies')

# Now you have a sampled dataframe with 0.1 percent of the original data, and you can proceed with your analysis using this sampled data.

In [ ]:
sampled_df.tail(50)

In [ ]:
# Create user-item matrix
matrix = sampled_df.pivot_table(index='userId', columns='title', values='rating')

In [ ]:
matrix.head(5)

In [ ]:
# Normalize user-item matrix
matrix_norm = matrix.subtract(matrix.mean(axis=1), axis = 'rows')

In [ ]:
# User similarity matrix using cosine similarity
user_similarity_cosine = cosine_similarity(matrix_norm.fillna(0))

In [ ]:
# User similarity matrix using Pearson correlation
user_similarity = matrix_norm.T.corr()
user_similarity.head()

In [ ]:
user_similarity_cosine

In [ ]:

# Find the number of unique values in the user similarity matrix
num_unique_values = user_similarity.nunique().sum()

print("Number of unique values in the user similarity matrix:", num_unique_values)

In [ ]:
# Transpose the data frame matrix (swap rows and columns)
df_transpose = matrix.transpose()


In [ ]:
users = list(df_transpose.columns)
movies = list(df_transpose.index)

In [ ]:
def get_rated_user_for_a_movie(df_transpose: pd.DataFrame, movie: str):
    return df_transpose.loc[title, :].dropna().index.values


def get_top_neighbors(
    user_similarity: pd.DataFrame, user: str, rated_users: str, n_neighbors: int
):
    return user_similarity[user][rated_users].nlargest(n_neighbors).to_dict()


def subtract_bias(rating: float, mean_rating: float):
    return rating - mean_rating


def get_neighbor_rating_without_bias_per_movie(
    ratings_df: pd.DataFrame, user: str, movie: str
):
    """Substract the rating of a user from the mean rating of that user to eliminate bias"""
    mean_rating = ratings_df[user].mean()
    rating = df_transpose.loc[movie, user]
    return subtract_bias(rating, mean_rating)


def get_ratings_of_neighbors(df_transpose: pd.DataFrame, neighbors: list, movie: str):
    """Get the ratings of all neighbors after adjusting for biases"""
    return [
        get_neighbor_rating_without_bias_per_movie(df_transpose, neighbor, movie)
        for neighbor in neighbors
    ]

def get_weighted_average_rating_of_neighbors(ratings: list, neighbor_distance: list):
    weighted_sum = np.array(ratings).dot(np.array(neighbor_distance))
    abs_neigbor_distance = np.abs(neighbor_distance)
    return weighted_sum / np.sum(abs_neigbor_distance)


def ger_user_rating(df_transpose: pd.DataFrame, user: str, avg_neighbor_rating: float):
    user_avg_rating = df_transpose[user].mean()
    return round(user_avg_rating + avg_neighbor_rating, 2)

In [ ]:
def predict_rating(
    df: pd.DataFrame,
    user_similarity: pd.DataFrame,
    user: str,
    movie: str,
    n_neighbors: int = 2,
):
    """Predict the rating of a user for a movie based on the ratings of neighbors"""
    df_transpose = df.copy()

    rated_users = get_rated_user_for_a_movie(df_transpose, title)

    top_neighbors_distance = get_top_neighbors(
        similarity_df, user, rated_users, n_neighbors
    )
    neighbors, distance = top_neighbors_distance.keys(), top_neighbors_distance.values()

    print(f"Top {n_neighbors} neighbors of user {user}, {title}: {list(neighbors)}")

    ratings = get_ratings_of_neighbors(df_transpose, neighbors, title)
    avg_neighbor_rating = get_weighted_average_rating_of_neighbors(
        ratings, list(distance)
    )

    return ger_user_rating(df_transpose, user, avg_neighbor_rating)

In [ ]:
# Convert the df_GT100 to a Surprise Dataset
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df_GT100[['userId', 'movieId', 'rating']], reader)

# Split the data into train and test sets (80% for training, 20% for testing)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Build and train the SVD model on the train_data
svd_model = SVD(n_factors=100, random_state=42)
svd_model.fit(train_data)

In [ ]:

def predict_rating_svd(svd_model, user, movie):
    """Predict the rating of a user for a movie using SVD"""
    prediction = svd_model.predict(user, movie)
    return prediction.est


In [ ]:
def predict_rating_combined(df, user_similarity, svd_model, user, movie, n_neighbors=2):
    """Predict the rating of a user for a movie using a combination of user-based collaborative filtering and SVD"""
    rated_users = get_rated_user_for_a_movie(df_transpose, movie)

    top_neighbors_distance = get_top_neighbors(
        user_similarity, user, rated_users, n_neighbors
    )
    neighbors, distance = top_neighbors_distance.keys(), top_neighbors_distance.values()

    print(f"Top {n_neighbors} neighbors of user {user} for {movie}: {list(neighbors)}")

    ratings = get_ratings_of_neighbors(df_transpose, neighbors, movie)
    avg_neighbor_rating = get_weighted_average_rating_of_neighbors(
        ratings, list(distance)
    )

    # Predict rating using SVD for the same movie
    svd_rating = predict_rating_svd(svd_model, user, movie)

    # Combine user-based CF and SVD predictions
    combined_rating = (0.7 * avg_neighbor_rating) + (0.3 * svd_rating)

    return ger_user_rating(df_transpose, user, combined_rating)

In [ ]:
def predict_rating_combined(df, user_similarity, svd_model, user, movie, n_neighbors=2):
    """Predict the rating of a user for a movie using a combination of user-based collaborative filtering and SVD"""
    rated_users = get_rated_user_for_a_movie(df_transpose, movie)

    top_neighbors_distance = get_top_neighbors(
        user_similarity, user, rated_users, n_neighbors
    )
    neighbors, distance = top_neighbors_distance.keys(), top_neighbors_distance.values()

    print(f"Top {n_neighbors} neighbors of user {user} for {movie}: {list(neighbors)}")

    ratings = get_ratings_of_neighbors(df_transpose, neighbors, movie)
    avg_neighbor_rating = get_weighted_average_rating_of_neighbors(
        ratings, list(distance)
    )

    # Predict rating using SVD for the same movie
    svd_rating = predict_rating_svd(svd_model, user, movie)

    # Combine user-based CF and SVD predictions
    combined_rating = (0.7 * avg_neighbor_rating) + (0.3 * svd_rating)

    return ger_user_rating(df_transpose, user, combined_rating)

In [ ]:
# Iterate through each row in the test dataframe
for index, row in test.iterrows():
    user = str(row['userId'])
    movie = str(row['movieId'])
    # Predict the rating using the combined prediction function
    rating = predict_rating_combined(df_GT100, user_similarity, svd_model, user, movie)
    # Append the user, movie, and predicted rating to the list
    predicted_ratings.append([user, movie, rating])



In [ ]:
# Create a DataFrame from the predicted ratings list
submission_df = pd.DataFrame(predicted_ratings, columns=['userId', 'movieId', 'rating'])

# Save the submission DataFrame to a CSV file for submission to Kaggle
submission_df.to_csv('submission.csv', index=False)